# Example Code for Implement the Noise Model

# Implement Noise Model

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
from qiskit.tools.jupyter import *
import warnings
# warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals

In [2]:
from qiskit.providers.fake_provider import *

In [3]:
import pickle

In [4]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model = pickle.load(file)

In [5]:
noise_model1 = noise.NoiseModel()

In [6]:
noise_modelreal = noise_model1.from_dict(noise_model)

/tmp/ipykernel_84400/404796782.py:1: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_modelreal = noise_model1.from_dict(noise_model)


In [7]:
noise_modelreal

<NoiseModel on ['', 'reset', 'id', 'sx', 'cx', 'x', 'measure']>

In [8]:
noise_modelreal.to_dict();

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as set of seeds to test results on your side, and the result will be calculated as the average of results from some of those seeds with some hidden seeds. And please use shots as 6000.

In [9]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
shot = 6000

## Hamiltonian conversion

By using SHVQE, we need to apply the inversal of Heisenberg part to the original Hamiltonian. First, load the original Hamiltonian from `Hamiltonian/OHhamiltonian.txt`.

In [10]:
with open('Hamiltonian/OHhamiltonian.txt', 'r') as file:
    pauli_text_lines = file.readlines()
paulis = []
weight = []
for line in pauli_text_lines:
    line = line.replace(' ', '')
    coeff, pauli_text_string = line.split("*")
    coeff = float(coeff)
    weight.append(coeff)
    pauli_text_string = pauli_text_string[::-1] # TODO: this is a quick but wrong fix for qubits order
    pauli_text_string = pauli_text_string.replace('\n', '')
    paulis.append(pauli_text_string)

Then, we use `Stim` to apply the inversal of Heisenberg part to Hamiltonian. In our protocal, the Heisenberg part consists of `cz` gates, which commute with each other. The inversal of the Heisenberg part is itself.

In [11]:
import stim

Load Heisenberg circuit.

In [12]:
heisenberg_text_combined = ''

with open('saved_models/shvqe_clifford_ncz10_hei.qasm', 'r') as file:
    heisenberg_text_lines = file.readlines()
    heisenberg_text_combined = ''.join(heisenberg_text_lines[3:]) # omit the header

In [13]:
import re


# Define the regular expression pattern
pattern = r"q\[(\d+)\],q\[(\d+)\]"

# Define the replacement string
replacement = r"\1 \2"

# Use the sub() function to replace the matches
stim_text = re.sub(pattern, replacement, heisenberg_text_combined).replace(';', '')

Apply the Heisenberg circuit to each Pauli string.

In [14]:
heisenberg_circ = stim.Circuit(stim_text)
paulis_new = []

for p in paulis:
    ps  = stim.PauliString(p.replace('I', '_'))
    ps = ps.after(heisenberg_circ)
    paulis_new.append(ps.__str__().replace('_', 'I'))

Construct the new Hamiltonian as observable. The number of Pauli strings is not changed by Clifford.

In [15]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp(paulis_new, weight)
print(f">>> Observable size: {observable.size}")

>>> Observable size: 631


# Circuit Transpile and Execution

In [16]:
from qiskit import qasm2
from qiskit_aer.primitives import Estimator

circuit = qasm2.load('saved_models/shvqe_clifford_ncz10_sch.qasm')

In [17]:
circuit.draw();

Transpile the circuit based on given system model from IBMQ_Montreal.

In [18]:
system_model = FakeMontreal()

In [19]:
transpiled_circuit = transpile(circuit, backend=system_model)

In [20]:
transpiled_circuit.draw();

Evaluate on the qiskit Estimator. Consider noiseless first.

In [21]:
# estimator = Estimator(
#     backend_options = {
#         'method': 'statevector',
#         'device': 'CPU',
#         'max_parallel_threads' : 0,
#         # 'max_parallel_experiments' : 0,
#         'max_parallel_shots' : 0,
#         'statevector_parallel_threshold' : 5,
#         'noise_model': None # noiseless
#     },
#     run_options = {
#         # 'shots': shot,
#         'seed': seeds,
#     },
#     skip_transpilation=False
# )

estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        # 'max_parallel_experiments' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'noise_model': None # noiseless
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    skip_transpilation=False
)

In [22]:
job = estimator.run(circuit, observable)
result = job.result()
print(f">>> {result}")

>>> EstimatorResult(values=array([-78.74530596]), metadata=[{'shots': 6000, 'variance': 1.454405338607831, 'simulator_metadata': [{'noise': 'ideal', 'batched_shots_optimization': False, 'measure_sampling': True, 'parallel_shots': 1, 'remapped_qubits': False, 'active_input_qubits': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'num_clbits': 12, 'parallel_state_update': 192, 'sample_measure_time': 0.040879375, 'num_qubits': 12, 'device': 'CPU', 'input_qubit_map': [[11, 11], [10, 10], [9, 9], [8, 8], [7, 7], [6, 6], [5, 5], [4, 4], [3, 3], [2, 2], [1, 1], [0, 0]], 'method': 'statevector', 'fusion': {'applied': False, 'max_fused_qubits': 5, 'threshold': 14, 'enabled': True}}, {'noise': 'ideal', 'batched_shots_optimization': False, 'measure_sampling': True, 'parallel_shots': 1, 'remapped_qubits': False, 'active_input_qubits': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'num_clbits': 12, 'parallel_state_update': 192, 'sample_measure_time': 0.065115684, 'num_qubits': 12, 'device': 'CPU', 'input_qubit_m

In [25]:
result.values

array([-78.74530596])

In [26]:
ref_value = -74.38714627
nuclear_repulsion_energy = 4.365374966545
result = result.values + nuclear_repulsion_energy
error_rate = abs(abs(ref_value - result) / ref_value * 100)

error_rate

array([0.00969962])